In [1]:
from src.data.preprocess_dataset import create_dataloader
from src.models.model import HGNModel, Validation
from src.models.document_retrieval import DocumentRetrieval
import torch
import json
import os

/opt/conda/envs/thesis/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
data_path = 'data/'
model_path = 'models'

print("Preprocessing data")
hotpotqa_path = 'external/'
print("Loading HotpotQA")
#hotpot_dev_distractor_v1
with open(os.path.join(data_path, hotpotqa_path, "hotpot_dev_distractor_v1.json"), "r") as f:
    hotpot = json.load(f)
hotpot = hotpot

Preprocessing data
Loading HotpotQA


In [3]:
# device = 'cuda'
# doc_retr_model_path = 'models/giwon_doc_retrieval/'
# print("Loading the document retrieval model")
# doc_retr = DocumentRetrieval(device, doc_retr_model_path)
# print("Computing the relevant documents")
# dict_ins2dict_doc2pred = doc_retr.predict_relevant_docs(hotpot)

In [4]:
# with open('data/interim/inference/dict_ins2dict_doc2pred_giwon.json', 'w+') as f:
#     json.dump(dict_ins2dict_doc2pred, f)

In [5]:
with open('data/interim/inference/dict_ins2dict_doc2pred_giwon.json', 'r') as f:
    dict_ins2dict_doc2pred1 = json.load(f)

In [6]:
dict_ins2dict_doc2pred = dict()
for k, v in dict_ins2dict_doc2pred1.items():
    dict_doc2pred = dict()
    for kk, vv in v.items():
        dict_doc2pred[int(kk)] = vv
    dict_ins2dict_doc2pred[int(k)] = dict_doc2pred

In [7]:
from src.data.graph_creation import Dataset
from src.data.preprocessing import NER_stanza
from src.data.preprocessing import SRL

In [8]:
# ner = NER_stanza()
# srl = SRL()
# list_ent_query = ner.extract_named_entities_from_query(hotpot)
# print("Extracting named entities")
# list_hotpot_ner = ner.extract_named_entities(hotpot, dict_ins2dict_doc2pred)
# print("Extracting SRL arguments from the query")
# dict_ins_query_srl_triples = srl.extract_srl_from_query(hotpot)
# print("Extracting SRL arguments")
# dict_ins_doc_sent_srl_triples = srl.extract_srl(hotpot, dict_ins2dict_doc2pred)
# print("Data loaded. Creating graphs")

In [9]:
# import pickle
# with open('data/interim/inference/list_ent_query_giwon.p', 'wb+') as f:
#     pickle.dump(list_ent_query, f)
# with open('data/interim/inference/list_hotpot_ner_giwon.p', 'wb+') as f:
#     pickle.dump(list_hotpot_ner, f)
# with open('data/interim/inference/dict_ins_query_srl_triples_giwon.p', 'wb+') as f:
#     pickle.dump(dict_ins_query_srl_triples, f)
# with open('data/interim/inference/dict_ins_doc_sent_srl_triples_giwon.p', 'wb+') as f:
#     pickle.dump(dict_ins_doc_sent_srl_triples, f)

In [10]:
import pickle
with open('data/interim/inference/list_ent_query_giwon.p', 'rb') as f:
    list_ent_query = pickle.load(f)
with open('data/interim/inference/list_hotpot_ner_giwon.p', 'rb') as f:
    list_hotpot_ner = pickle.load(f)
with open('data/interim/inference/dict_ins_query_srl_triples_giwon.p', 'rb') as f:
    dict_ins_query_srl_triples = pickle.load(f)
with open('data/interim/inference/dict_ins_doc_sent_srl_triples_giwon.p', 'rb') as f:
    dict_ins_doc_sent_srl_triples = pickle.load(f)

In [11]:
pretrained_weights = 'bert-base-cased'
train_dataset = Dataset(hotpot, list_hotpot_ner, dict_ins_doc_sent_srl_triples,
                            dict_ins_query_srl_triples, list_ent_query, 
                            dict_ins2dict_doc2pred=dict_ins2dict_doc2pred, batch_size=1,
                            pretrained_weights=pretrained_weights)

In [12]:
(list_graphs, list_context, list_span_idx) = train_dataset.create_dataloader()

100%|██████████| 7405/7405 [04:00<00:00, 30.83it/s]


In [13]:
list_input_ids = [context['input_ids'] for context in list_context]
list_token_type_ids = [context['token_type_ids'] for context in list_context]
list_attention_masks = [context['attention_mask'] for context in list_context]

tensor_input_ids = torch.tensor(list_input_ids)
tensor_token_type_ids = torch.tensor(list_token_type_ids)
tensor_attention_masks = torch.tensor(list_attention_masks)

In [14]:
from src.models.model import HGNModel, Validation

In [15]:
model_path = 'models/no_srl/'
model = HGNModel.from_pretrained(model_path)
model.cuda()

/opt/conda/envs/thesis/lib/python3.6/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


HGNModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [16]:
validation = Validation(model, hotpot, list_graphs,
                        tensor_input_ids, tensor_attention_masks,
                        tensor_token_type_ids)
preds = validation.get_answer_predictions(dict_ins2dict_doc2pred)

100%|██████████| 7405/7405 [25:39<00:00,  4.81it/s]


In [17]:
import json

In [18]:
with open('preds_no_srl_top2.json', 'w+') as f:
    json.dump(preds, f)